In [1]:
from seeq import spy
import pandas as pd

# Set the compatibility option so that you maximize the chance that SPy will remain compatible with your notebook/script
spy.options.compatibility = 193

In [2]:
# Log into Seeq Server if you're not using Seeq Data Lab:
spy.login(url='http://localhost:34216', credentials_file='../credentials.key', force=False)

# Asset Trees 2: Templates

In the [first asset trees tutorial notebook](Asset%20Trees%201%20-%20Introduction.ipynb), we learned how to use the `spy.assets.Tree` Python class to define, modify, and push asset trees to Seeq. In this notebook, we will dive deeper into the `spy.assets` submodule and explore its ability to create asset trees out of customized Python "templates".

You may have heard of the concept of a [_Digital Twin_](https://en.wikipedia.org/wiki/Digital_twin), which is a "virtual representation that serves as the real-time digital counterpart of a physical object or process." The `spy.assets` submodule provides a framework for defining Digital Twins using the power and relative ease-of-use of the Python programming language. You can leverage object-oriented design principles like _encapsulation_, _inheritance_, _composition_, and _mixins_ to increase reuse and consistency while still accommodating the many exceptions that naturally occur in manufacturing scenarios.

As with other capabilities in SPy, the work you do with `spy.assets` is (by default) sandboxed to a particular workbook that you own. As a result, you can easily experiment with new asset structures and calculations before publishing to your broader organization.

## Simple Trees vs Templates

When should you use the `spy.assets.Tree()` and when should you use the Template methods described here?

Start with the simpler `spy.assets.Tree()` functionality unless, as you skim through this documentation, it's clear that you'd like to try the Template functionality first. You can always "graduate" to this more advanced approach.

## Concepts

There are several important concepts to understand as we dive in to the `spy.assets` submodule:

* An **asset** is simply a container (with associated _properties_) that has children. Those children can be other assets and/or _attributes_ (signals, conditions and scalars).
* An **attribute** is a signal, condition or scalar that is a child of an asset and is said to be _contained_ by that asset. This is a similar concept to OSIsoft Asset Framework's _attribute_.
* A **property** is a named value (with optional units of measure) that captures metadata associated with assets, signals, conditions and scalars. They correspond to the columns in a metadata DataFrame.
* A **component** is an asset that is a child of another asset and is said to be _contained_ by the parent asset. For example, a _Furnace_ asset may contain a _Blast Air Blower_.
* A **template** defines the set of _attributes_ and _components_ that comprise a particular Asset. A template corresponds to a class of asset like _Furnace_, _Well_, _Pump_.
* A **reference** is a link to a signal, condition or scalar that exists somewhere else. If you are mapping a flat list of tags into an asset hierarchy, the asset hierarchy contains _references_ to those tags.

## How Python fits in

A template is specified in Python by defining a [class](https://docs.python.org/3/tutorial/classes.html) that derives from `spy.assets.Asset` or `spy.assets.Mixin` (more about _mixins_ later). These classes can have member functions and data members, just like any other Python class.

Classes can have special member functions that represent _attributes_. They are _decorated_ with the `spy.assets.Asset.Attribute` decorator, which tells the SPy framework to use them to define attributes for the asset. Similarly, the `spy.assets.Asset.Component` decorated indicates to the SPy framework that a _component_ is being defined as a child of the asset, which is usually a different Python class instance that derives from `spy.assets.Asset` or `spy.assets.Mixin`.

You'll see how it works in the examples that follow.

## Getting Started

First we need to import the Python modules we will need and log in:

In [3]:
from seeq.spy.assets import Asset, ItemGroup

# Show all data in DataFrame output -- don't truncate it
pd.set_option('display.max_colwidth', None)

## Preparing the Ingredients

There are two main steps to creating an asset tree, and they're similar to cooking in your kitchen: First you find and prepare the "ingredients", then you use them in a "recipe".

The "ingredients" are the signals, conditions and/or scalars that already exist in Seeq, either as indexed items from external datasources or as items you've imported to the internal Seeq database. You will create a DataFrame that represents the _metadata_ ingredients, and those ingredients will be used by `spy.assets.build()` to create another DataFrame full of new items to be pushed as an asset tree.

For this example, we're going to map a flat tag structure into an asset template called `HVAC`. We will use Seeq's built-in Example Data.

Let's search for all the "flat list" example data tags that have the pattern `Area <letter>_<sensor name>`:

In [4]:
hvac_metadata_df = spy.search({
    'Name': 'Area ?_*',
    'Type':'StoredSignal',
    'Datasource Name':'Example Data'
})

hvac_metadata_df.head()

,Name,Time,Count,Pages,Result
0,Area ?_*,00:00:00.13,68,1,Success


,ID,Name,Description,Type,Value Unit Of Measure,Datasource Name,Archived
0,0EF58FC2-808C-7140-8131-94452FC3710A,Area G_Temperature,NaN,StoredSignal,°F,Example Data,False
1,0EF58FC2-819B-7130-8DA5-CEAC2086E025,Area G_Wet Bulb,NaN,StoredSignal,°F,Example Data,False
2,0EF58FC2-81CE-7580-85BE-080C7622D9BE,Area G_Relative Humidity,NaN,StoredSignal,%,Example Data,False
3,0EF58FC2-81D8-71C0-8CDC-302A3AE0ED26,Area C_Optimizer,NaN,StoredSignal,NaN,Example Data,False
4,0EF58FC2-8206-77F0-8519-ECD251B886A6,Area A_Temperature,NaN,StoredSignal,°F,Example Data,False


The `hvac_metadata_df` _metadata_ DataFrame serves as the "ingredients" for the recipe that we will define a little later.

In order to build the HVAC assets, we must add three important columns to our _metadata_ DataFrame:

* `Build Asset` specifies the name of the asset that a row of metadata applies to. In our case, that will be `Area X`, where X is a letter differentiating the different areas of the plant that an HVAC system serves.
* `Build Path` specifies the path through the asset hierarchy where you want the asset to live.

We will create these columns using the power of Pandas DataFrames:

In [5]:
# We can use Pandas' string extraction capabilities to create the Build Asset column
hvac_metadata_df['Build Asset'] = hvac_metadata_df['Name'].str.extract('(Area .)_.*')

# We will specify a simple path in a new tree where we want these to live
hvac_metadata_df['Build Path'] = 'My HVAC Units >> Facility #1'

hvac_metadata_df

,ID,Name,Description,Type,Value Unit Of Measure,Datasource Name,Archived,Build Asset,Build Path
0,0EF58FC2-808C-7140-8131-94452FC3710A,Area G_Temperature,NaN,StoredSignal,°F,Example Data,False,Area G,My HVAC Units >> Facility #1
1,0EF58FC2-819B-7130-8DA5-CEAC2086E025,Area G_Wet Bulb,NaN,StoredSignal,°F,Example Data,False,Area G,My HVAC Units >> Facility #1
2,0EF58FC2-81CE-7580-85BE-080C7622D9BE,Area G_Relative Humidity,NaN,StoredSignal,%,Example Data,False,Area G,My HVAC Units >> Facility #1
3,0EF58FC2-81D8-71C0-8CDC-302A3AE0ED26,Area C_Optimizer,NaN,StoredSignal,NaN,Example Data,False,Area C,My HVAC Units >> Facility #1
4,0EF58FC2-8206-77F0-8519-ECD251B886A6,Area A_Temperature,NaN,StoredSignal,°F,Example Data,False,Area A,My HVAC Units >> Facility #1
...,...,...,...,...,...,...,...,...,...
63,0EF58FC2-8C3F-EAF0-A54D-2EDBDA836AD2,Area G_Compressor Stage,NaN,StoredSignal,string,Example Data,False,Area G,My HVAC Units >> Facility #1
64,0EF58FC2-8C7F-6290-BEE8-1BE39219249E,Area A_Compressor Power,NaN,StoredSignal,kW,Example Data,False,Area A,My HVAC Units >> Facility #1
65,0EF58FC2-8C97-F930-B81A-D74F9F8BB2FB,Area B_Wet Bulb,NaN,StoredSignal,°F,Example Data,False,Area B,My HVAC Units >> Facility #1
66,0EF58FC2-8D14-7160-9EE1-1EC3CA6EFB2C,Area E_Temperature,NaN,StoredSignal,°F,Example Data,False,Area E,My HVAC Units >> Facility #1


## Writing the Recipe

The recipe that turns the ingredients into an asset structure is specified using Python classes that derive from `Asset` or `Mixin`.

First let's define our `HVAC` class:

In [6]:
class HVAC(Asset):
    
    @Asset.Attribute()
    def Temperature(self, metadata):
        # We use simple Pandas syntax to select for a row in the DataFrame corresponding to our desired tag
        return metadata[metadata['Name'].str.endswith('Temperature')]
    
    @Asset.Attribute()
    def Relative_Humidity(self, metadata):
        # All Attribute functions must take (self, metadata) as parameters
        return metadata[metadata['Name'].str.contains('Humidity')]

This Python code defines an `HVAC` asset that has two attributes: `Temperature` and `Relative Humidity`. These attributes are represented in Python as functions that can return any of the following:

1. A single-row DataFrame containing an ID column that identifies an item (signal/condition/scalar/metric) to expose on this asset. (As seen above.)
2. A dictionary that defines the item. A `Type` entry is required (whose value must be `Signal`, `Condition`, `Scalar` or `Metric`). If `Name` is not supplied, the function name will be used as the `Name` with any underscores automatically replaced with a space. As you'll see below, you can specify `Formula` and `Formula Parameters` if you are trying to specify a calculated item.
3. A _list_ of dictionaries if you want to specify multiple items in the format of (2) above.

Now we can feed the "ingredients" into the "recipe" using the `spy.assets.build()` function. This command will build a new set of asset and signal definitions based on the `hvac_metadata_df` _metadata_ DataFrame. Each unique combination of `Build Path` and `Build Asset` will be treated as a different asset, and the `metadata` argument that is passed in to the `Asset.Attribute()` decorated functions will only contain the rows for a particular `Build Path` / `Build Asset` pair.

In [7]:
build_df = spy.assets.build(HVAC, hvac_metadata_df)

build_df

# NOTE:
#
# There will be errors in this example, the status table will be colored red. Read more below.

,Build Path,Build Asset,Build Template,Build Result
0,My HVAC Units >> Facility #1,Area G,HVAC,Success
1,My HVAC Units >> Facility #1,Area C,HVAC,Success
2,My HVAC Units >> Facility #1,Area A,HVAC,Success
3,My HVAC Units >> Facility #1,Area I,HVAC,Success
4,My HVAC Units >> Facility #1,Area B,HVAC,Success
5,My HVAC Units >> Facility #1,Area H,HVAC,Success
6,My HVAC Units >> Facility #1,Area D,HVAC,Success
7,My HVAC Units >> Facility #1,Area J,HVAC,Success
8,My HVAC Units >> Facility #1,Area E,HVAC,Success
9,My HVAC Units >> Facility #1,Area K,HVAC,Success


,ID,Description,Type,Value Unit Of Measure,Datasource Name,Archived,Referenced Name,Reference,Name,Asset,Asset Object,Path,Template,Build Result
0,0EF58FC2-81CE-7580-85BE-080C7622D9BE,NaN,StoredSignal,%,Example Data,False,Area G_Relative Humidity,True,Relative Humidity,Area G,My HVAC Units >> Facility #1 >> Area G,My HVAC Units >> Facility #1,HVAC,Success
1,0EF58FC2-808C-7140-8131-94452FC3710A,NaN,StoredSignal,°F,Example Data,False,Area G_Temperature,True,Temperature,Area G,My HVAC Units >> Facility #1 >> Area G,My HVAC Units >> Facility #1,HVAC,Success
2,NaN,NaN,Asset,NaN,NaN,NaN,NaN,NaN,Area G,Area G,My HVAC Units >> Facility #1 >> Area G,My HVAC Units >> Facility #1,HVAC,Success
3,0EF58FC2-8245-FF90-99E6-9CE8E5FF779E,NaN,StoredSignal,%,Example Data,False,Area C_Relative Humidity,True,Relative Humidity,Area C,My HVAC Units >> Facility #1 >> Area C,My HVAC Units >> Facility #1,HVAC,Success
4,0EF58FC2-8A5C-6490-85CE-FD5BD9A25B54,NaN,StoredSignal,°F,Example Data,False,Area C_Temperature,True,Temperature,Area C,My HVAC Units >> Facility #1 >> Area C,My HVAC Units >> Facility #1,HVAC,Success
5,NaN,NaN,Asset,NaN,NaN,NaN,NaN,NaN,Area C,Area C,My HVAC Units >> Facility #1 >> Area C,My HVAC Units >> Facility #1,HVAC,Success
6,0EF58FC2-83AA-66B0-8B9F-E021808A2620,NaN,StoredSignal,%,Example Data,False,Area A_Relative Humidity,True,Relative Humidity,Area A,My HVAC Units >> Facility #1 >> Area A,My HVAC Units >> Facility #1,HVAC,Success
7,0EF58FC2-8206-77F0-8519-ECD251B886A6,NaN,StoredSignal,°F,Example Data,False,Area A_Temperature,True,Temperature,Area A,My HVAC Units >> Facility #1 >> Area A,My HVAC Units >> Facility #1,HVAC,Success
8,NaN,NaN,Asset,NaN,NaN,NaN,NaN,NaN,Area A,Area A,My HVAC Units >> Facility #1 >> Area A,My HVAC Units >> Facility #1,HVAC,Success
9,0EF58FC2-8642-77B0-BBFB-7F069389F904,NaN,StoredSignal,%,Example Data,False,Area I_Relative Humidity,True,Relative Humidity,Area I,My HVAC Units >> Facility #1 >> Area I,My HVAC Units >> Facility #1,HVAC,Success


In the progress table above (which should be colored red), if you look at the `Build Result` column for Area F, you can see that no matching metadata was found. If you then do a search in Seeq Workbench for `Area F_`, you'll see that there are no `Temperature` or `Relative Humidity` tags for that area. That's fine! When we push, we simply won't add signals for Area F.

The new `build_df` DataFrame contains new metadata that represent all the Seeq items that can be pushed into Seeq to realize this simple asset model.

In [8]:
spy.push(metadata=build_df, workbook='SPy Documentation Examples >> spy.assets')

,Description,Type,Datasource Name,Archived,Referenced Name,Reference,Name,Asset,Asset Object,Path,...,Formula Parameters,Cache Enabled,Referenced ID,Scoped To,Datasource Class,Datasource ID,Data ID,ID,Push Result,Value Unit Of Measure
0,NaN,CalculatedSignal,Example Data,False,Area G_Relative Humidity,True,Relative Humidity,Area G,My HVAC Units >> Facility #1 >> Area G,My HVAC Units >> Facility #1,...,[signal=0EF58FC2-81CE-7580-85BE-080C7622D9BE],False,0EF58FC2-81CE-7580-85BE-080C7622D9BE,0EF5BE2D-2923-FB30-A567-172B344E74EE,Seeq Data Lab,Seeq Data Lab,[0EF5BE2D-2923-FB30-A567-172B344E74EE] {Signal} My HVAC Units >> Facility #1 >> Area G >> Relative Humidity,0EF5BE2D-3FAA-64D0-87B1-A06BC7F0605E,Success,NaN
1,NaN,CalculatedSignal,Example Data,False,Area G_Temperature,True,Temperature,Area G,My HVAC Units >> Facility #1 >> Area G,My HVAC Units >> Facility #1,...,[signal=0EF58FC2-808C-7140-8131-94452FC3710A],False,0EF58FC2-808C-7140-8131-94452FC3710A,0EF5BE2D-2923-FB30-A567-172B344E74EE,Seeq Data Lab,Seeq Data Lab,[0EF5BE2D-2923-FB30-A567-172B344E74EE] {Signal} My HVAC Units >> Facility #1 >> Area G >> Temperature,0EF5BE2D-4127-6290-8CF3-FC4399F681BD,Success,NaN
2,NaN,Asset,NaN,NaN,NaN,NaN,Area G,Area G,My HVAC Units >> Facility #1 >> Area G,My HVAC Units >> Facility #1,...,NaN,NaN,NaN,0EF5BE2D-2923-FB30-A567-172B344E74EE,Seeq Data Lab,Seeq Data Lab,[0EF5BE2D-2923-FB30-A567-172B344E74EE] {Asset} My HVAC Units >> Facility #1 >> Area G,0EF5BE2D-4888-ECF0-A176-90C60EE8D980,Success,NaN
3,NaN,CalculatedSignal,Example Data,False,Area C_Relative Humidity,True,Relative Humidity,Area C,My HVAC Units >> Facility #1 >> Area C,My HVAC Units >> Facility #1,...,[signal=0EF58FC2-8245-FF90-99E6-9CE8E5FF779E],False,0EF58FC2-8245-FF90-99E6-9CE8E5FF779E,0EF5BE2D-2923-FB30-A567-172B344E74EE,Seeq Data Lab,Seeq Data Lab,[0EF5BE2D-2923-FB30-A567-172B344E74EE] {Signal} My HVAC Units >> Facility #1 >> Area C >> Relative Humidity,0EF5BE2D-41C0-FF80-A9A4-AD8E56F9013E,Success,NaN
4,NaN,CalculatedSignal,Example Data,False,Area C_Temperature,True,Temperature,Area C,My HVAC Units >> Facility #1 >> Area C,My HVAC Units >> Facility #1,...,[signal=0EF58FC2-8A5C-6490-85CE-FD5BD9A25B54],False,0EF58FC2-8A5C-6490-85CE-FD5BD9A25B54,0EF5BE2D-2923-FB30-A567-172B344E74EE,Seeq Data Lab,Seeq Data Lab,[0EF5BE2D-2923-FB30-A567-172B344E74EE] {Signal} My HVAC Units >> Facility #1 >> Area C >> Temperature,0EF5BE2D-4225-7110-A6AB-14BE086C78CB,Success,NaN
5,NaN,Asset,NaN,NaN,NaN,NaN,Area C,Area C,My HVAC Units >> Facility #1 >> Area C,My HVAC Units >> Facility #1,...,NaN,NaN,NaN,0EF5BE2D-2923-FB30-A567-172B344E74EE,Seeq Data Lab,Seeq Data Lab,[0EF5BE2D-2923-FB30-A567-172B344E74EE] {Asset} My HVAC Units >> Facility #1 >> Area C,0EF5BE2D-4888-ECF0-BBD6-64406C031887,Success,NaN
6,NaN,CalculatedSignal,Example Data,False,Area A_Relative Humidity,True,Relative Humidity,Area A,My HVAC Units >> Facility #1 >> Area A,My HVAC Units >> Facility #1,...,[signal=0EF58FC2-83AA-66B0-8B9F-E021808A2620],False,0EF58FC2-83AA-66B0-8B9F-E021808A2620,0EF5BE2D-2923-FB30-A567-172B344E74EE,Seeq Data Lab,Seeq Data Lab,[0EF5BE2D-2923-FB30-A567-172B344E74EE] {Signal} My HVAC Units >> Facility #1 >> Area A >> Relative Humidity,0EF5BE2D-428B-F9B0-983F-C6C713479D59,Success,NaN
7,NaN,CalculatedSignal,Example Data,False,Area A_Temperature,True,Temperature,Area A,My HVAC Units >> Facility #1 >> Area A,My HVAC Units >> Facility #1,...,[signal=0EF58FC2-8206-77F0-8519-ECD251B886A6],False,0EF58FC2-8206-77F0-8519-ECD251B886A6,0EF5BE2D-2923-FB30-A567-172B344E74EE,Seeq Data Lab,Seeq Data Lab,[0EF5BE2D-2923-FB30-A567-172B344E74EE] {Signal} My HVAC Units >> Facility #1 >> Area A >> Temperature,0EF5BE2D-42F9-7780-B2E8-119A05678A25,Success,NaN
8,NaN,Asset,NaN,NaN,NaN,NaN,Area A,Area A,My HVAC Units >> Facility #1 >> Area A,My HVAC Units >> Facility #1,...,NaN,NaN,NaN,0EF5BE2D-2923-FB30-A567-172B344E74EE,Seeq Data Lab,Seeq Data Lab,[0EF5BE2D-2923-FB30-A567-172B344E74EE] {Asset} My HVAC Units >> Facility #1 >> Area A,0EF5BE2D-4

The asset tree is now available for viewing in Seeq. It is scoped to a workbook that you own called _SPy Documentation Examples >> spy.assets_, so you won't see it in any other workbook. If/when you want to publish the tree globally, add the `workbook=None` parameter to your `spy.push()` function call.

## Calculated Signals, Conditions and Scalars

Now let's do something a little more advanced. We'll define a new class that has calculations alongside references to raw signals. This class will _derive from_ our existing `HVAC` class so that it _inherits_ the references we already defined.

In [9]:
class HVAC_With_Calcs(HVAC):
    
    @Asset.Attribute()
    def Temperature_Rate_Of_Change(self, metadata):
        return {
            'Type': 'Signal',
            
            # This formula will give us a nice derivative in F/h
            'Formula': '$temp.lowPassFilter(150min, 3min, 333).derivative() * 3600 s/h',
            
            'Formula Parameters': {
                # We can reference the base class' Temperature attribute here as a dependency
                '$temp': self.Temperature(),
            }
        }
    
    @Asset.Attribute()
    def Too_Hot(self, metadata):
        return {
            'Type': 'Condition',
            'Formula': '$temp.valueSearch(isGreaterThan($threshold))',
            'Formula Parameters': {
                '$temp': self.Temperature(),
                
                # We can also reference other attributes in this derived class
                '$threshold': self.Hot_Threshold()
            }
        }
    
    @Asset.Attribute()
    def Hot_Threshold(self, metadata):
        return {
            'Type': 'Scalar',
            'Formula': '80F'
        }

We'll make some adjustments to our metadata DataFrame to use the new template and then push into Seeq:

In [10]:
# Make a copy of our original metadata but change the Build Template
hvac_with_calcs_metadata_df = hvac_metadata_df.copy()

build_with_calcs_df = spy.assets.build(HVAC_With_Calcs, hvac_with_calcs_metadata_df)

pd.set_option('display.max_colwidth', 50)

spy.push(metadata=build_with_calcs_df, workbook='SPy Documentation Examples >> spy.assets')

,Type,Formula,Name,Asset,Asset Object,Path,Template,Build Result,ID,Description,...,Referenced Name,Reference,Formula Parameters,Scoped To,Datasource Class,Datasource ID,Data ID,Push Result,Cache Enabled,Referenced ID
0,CalculatedScalar,80F,Hot Threshold,Area G,My HVAC Units >> Facility #1 >> Area G,My HVAC Units >> Facility #1,HVAC With Calcs,Success,0EF5BE2D-5581-71F0-9A3C-E7EA8CD4761C,NaN,...,NaN,NaN,[],0EF5BE2D-2923-FB30-A567-172B344E74EE,Seeq Data Lab,Seeq Data Lab,[0EF5BE2D-2923-FB30-A567-172B344E74EE] {Scalar...,Success,NaN,NaN
1,CalculatedSignal,$signal.setUnits('%'),Relative Humidity,Area G,My HVAC Units >> Facility #1 >> Area G,My HVAC Units >> Facility #1,HVAC With Calcs,Success,0EF5BE2D-3FAA-64D0-87B1-A06BC7F0605E,NaN,...,Area G_Relative Humidity,True,[signal=0EF58FC2-81CE-7580-85BE-080C7622D9BE],0EF5BE2D-2923-FB30-A567-172B344E74EE,Seeq Data Lab,Seeq Data Lab,[0EF5BE2D-2923-FB30-A567-172B344E74EE] {Signal...,Success,False,0EF58FC2-81CE-7580-85BE-080C7622D9BE
2,CalculatedSignal,$signal.setUnits('°F'),Temperature,Area G,My HVAC Units >> Facility #1 >> Area G,My HVAC Units >> Facility #1,HVAC With Calcs,Success,0EF5BE2D-4127-6290-8CF3-FC4399F681BD,NaN,...,Area G_Temperature,True,[signal=0EF58FC2-808C-7140-8131-94452FC3710A],0EF5BE2D-2923-FB30-A567-172B344E74EE,Seeq Data Lab,Seeq Data Lab,[0EF5BE2D-2923-FB30-A567-172B344E74EE] {Signal...,Success,False,0EF58FC2-808C-7140-8131-94452FC3710A
3,CalculatedSignal,"$temp.lowPassFilter(150min, 3min, 333).derivat...",Temperature Rate Of Change,Area G,My HVAC Units >> Facility #1 >> Area G,My HVAC Units >> Facility #1,HVAC With Calcs,Success,0EF5BE2D-5819-62F0-A9D8-D57E66B4FDAE,NaN,...,NaN,NaN,[temp=0EF5BE2D-4127-6290-8CF3-FC4399F681BD],0EF5BE2D-2923-FB30-A567-172B344E74EE,Seeq Data Lab,Seeq Data Lab,[0EF5BE2D-2923-FB30-A567-172B344E74EE] {Signal...,Success,NaN,NaN
4,CalculatedCondition,$temp.valueSearch(isGreaterThan($threshold)),Too Hot,Area G,My HVAC Units >> Facility #1 >> Area G,My HVAC Units >> Facility #1,HVAC With Calcs,Success,0EF5BE2D-5C0E-75E0-8C20-DA80EA95535C,NaN,...,NaN,NaN,"[temp=0EF5BE2D-4127-6290-8CF3-FC4399F681BD, th...",0EF5BE2D-2923-FB30-A567-172B344E74EE,Seeq Data Lab,Seeq Data Lab,[0EF5BE2D-2923-FB30-A567-172B344E74EE] {Condit...,Success,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,CalculatedCondition,$temp.valueSearch(isGreaterThan($threshold)),Too Hot,Area Z,My HVAC Units >> Facility #1 >> Area Z,My HVAC Units >> Facility #1,HVAC With Calcs,Success,0EF5BE2D-5E9F-71B0-B25A-A0F8BF3EE2E5,NaN,...,NaN,NaN,"[temp=0EF5BE2D-481D-6630-9F70-78DE2E5D91FD, th...",0EF5BE2D-2923-FB30-A567-172B344E74EE,Seeq Data Lab,Seeq Data Lab,[0EF5BE2D-2923-FB30-A567-172B344E74EE] {Condit...,Success,NaN,NaN
66,Asset,NaN,Area Z,Area Z,My HVAC Units >> Facility #1 >> Area Z,My HVAC Units >> Facility #1,HVAC With Calcs,Success,0EF5BE2D-4895-6040-9E72-61B4139BEB68,NaN,...,NaN,NaN,NaN,0EF5BE2D-2923-FB30-A567-172B344E74EE,Seeq Data Lab,Seeq Data Lab,[0EF5BE2D-2923-FB30-A567-172B344E74EE] {Asset}...,Success,NaN,NaN
__side_effect_asset_1__,Asset,NaN,My HVAC Units,My HVAC Units,NaN,,NaN,NaN,0EF5BE2D-487C-F9A0-8334-4F3E5D06EFCE,NaN,...,NaN,NaN,NaN,NaN,Seeq Data Lab,Seeq Data Lab,[0EF5BE2D-2923-FB30-A567-172B344E74EE] {Asset}...,Success,NaN,NaN
__side_effect_asset_2__,Asset,NaN,Facility #1,Facility #1,NaN,My HVAC Units,NaN,NaN,0EF5BE2D-4886-75E0-86D7-557BE6494D53,NaN,...,NaN,NaN,NaN,NaN,Seeq Data Lab,Seeq Data Lab,[0EF5BE2D-2923-FB30-A567-172B344E74EE] {Asset}...,Success,NaN,NaN


Now when you look at the asset tree in Seeq, you'll see `Temperature Rate Of Change`, `Too Hot` and `Hot Threshold`. Each time we push, we are overwriting the definition we pushed previously.

## Components

You can create an entire asset hierarchy by defining Asset classes that are _composed_ of other Asset classes. To illustrate, let's start by preparing a DataFrame with our ingredients. This example will be a little contrived given the example data that is available in Seeq.

We will define a metadata DataFrame where there are two Refrigerators and each will have two Compressors. In the cell below, you will see that we define the `Build Asset` column with the Refigerator name but also define a `Compressor` column with the Compressor name. We have assigned Area A-D signals to specific Refrigerators and Compressors.

In [11]:
metadata_df = spy.search({
    'Name': r'/Area [A-D]_(?:Temperature|Compressor Power)/',
    'Type':'StoredSignal',
    'Datasource Name':'Example Data'
})

metadata_df.loc[metadata_df['Name'] == 'Area A_Temperature', 'Build Asset'] = 'Refrigerator 1'
metadata_df.loc[metadata_df['Name'] == 'Area A_Compressor Power', 'Build Asset'] = 'Refrigerator 1'
metadata_df.loc[metadata_df['Name'] == 'Area A_Compressor Power', 'Compressor'] = 'Compressor 1'
metadata_df.loc[metadata_df['Name'] == 'Area B_Compressor Power', 'Build Asset'] = 'Refrigerator 1'
metadata_df.loc[metadata_df['Name'] == 'Area B_Compressor Power', 'Compressor'] = 'Compressor 2'
metadata_df.loc[metadata_df['Name'] == 'Area C_Temperature', 'Build Asset'] = 'Refrigerator 2'
metadata_df.loc[metadata_df['Name'] == 'Area C_Compressor Power', 'Build Asset'] = 'Refrigerator 2'
metadata_df.loc[metadata_df['Name'] == 'Area C_Compressor Power', 'Compressor'] = 'Compressor 3'
metadata_df.loc[metadata_df['Name'] == 'Area D_Compressor Power', 'Build Asset'] = 'Refrigerator 2'
metadata_df.loc[metadata_df['Name'] == 'Area D_Compressor Power', 'Compressor'] = 'Compressor 4'

metadata_df['Build Path'] = 'Refrigerator Units'

metadata_df[['ID', 'Name', 'Build Path', 'Build Asset', 'Compressor']]

,Name,Datasource Class,Time,Count,Pages,Result
0,/Area [A-D]_(?:Temperature|Compressor Power)/,Time Series CSV Files,00:00:00.03,8,1,Success


,ID,Name,Build Path,Build Asset,Compressor
0,0EF58FC2-8206-77F0-8519-ECD251B886A6,Area A_Temperature,Refrigerator Units,Refrigerator 1,NaN
1,0EF58FC2-82AC-E830-9324-CD3E38A9EB6E,Area B_Temperature,Refrigerator Units,NaN,NaN
2,0EF58FC2-85A1-7590-AA5F-C66CA82059F8,Area B_Compressor Power,Refrigerator Units,Refrigerator 1,Compressor 2
3,0EF58FC2-8673-64F0-8C6A-4C5AE6D5CFC8,Area D_Compressor Power,Refrigerator Units,Refrigerator 2,Compressor 4
4,0EF58FC2-88FA-6480-92A3-70610FAED8C0,Area C_Compressor Power,Refrigerator Units,Refrigerator 2,Compressor 3
5,0EF58FC2-8A5C-6490-85CE-FD5BD9A25B54,Area C_Temperature,Refrigerator Units,Refrigerator 2,NaN
6,0EF58FC2-8B7C-75F0-99A3-B0D0420939F3,Area D_Temperature,Refrigerator Units,NaN,NaN
7,0EF58FC2-8C7F-6290-BEE8-1BE39219249E,Area A_Compressor Power,Refrigerator Units,Refrigerator 1,Compressor 1


Now we can define our Asset classes. Will be using the `Asset.Component()` decorator and the `self.build_components()` function:

In [12]:
class Refrigerator(Asset):
    @Asset.Attribute()
    def Temperature(self, metadata):
        # This signal attribute is assigned to the Refrigerator asset
        return metadata[metadata['Name'].str.endswith('Temperature')]

    # Note the use of Asset.Component here, which allows us to return a list of definitions
    # instead of just a single definition.
    @Asset.Component()
    def Compressors(self, metadata):
        # Using the Compressor template class, we build all of the compressor definitions
        # associated with a particular Refrigerator. The column_name supplied tells the
        # build_components function which metadata column to use for the Compressor names.
        return self.build_components(template=Compressor, metadata=metadata, column_name='Compressor')
    
    @Asset.Attribute()
    def Compressor_Power_Max(self, metadata):
        # We can refer to the Compressors and "pick" attributes for which to perform a
        # roll up. In this example, we're picking the 'Power' signals that are on each
        # compressor and creating a new signal representing the maximum power across
        # all the compressors.
        return self.Compressors().pick({
            'Name': 'Power'
        }).roll_up('maximum')
    
    @Asset.Attribute()
    def Compressor_High_Power(self, metadata):
        # Similar to Compressor_Power_Max, we are rolling up a compressor calculation but
        # this time it's a condition. 'High Power' at the Refrigerator level will have
        # capsules if either compressor's 'High Power' condition is present.
        #
        # This time we'll use a different method of picking the child items than we used
        # in Compressor_Power_Max() above. In this case, we're going to select the set of
        # compressors that are owned by this asset from the entire set of assets, and use
        # Python conditional logic to find the "High Power" conditions. What you see
        # below is called a Python "list comprehension" that combines iteration over all
        # assets (the "for/in" construct) with filtering (the "if" statement).
        #
        # Helpful functions:
        #  asset.is_child_of(self)      - Is the asset one of my direct children?
        #  asset.is_parent_of(self)     - Is the asset my direct parent?
        #  asset.is_descendant_of(self) - Is the asset below me in the tree?
        #  asset.is_ancestor_of(self)   - Is the asset above me? (i.e. parent/grandparent/great-grandparent/etc)
        #
        return ItemGroup([
            asset.High_Power() for asset in self.all_assets()
            if asset.is_child_of(self)
        ]).roll_up('union')
    
class Compressor(Asset):
    @Asset.Attribute()
    def Power(self, metadata):
        # Each compressor has just a single attribute, Power
        return metadata[metadata['Name'].str.endswith('Power')]
    
    @Asset.Attribute()
    def High_Power(self, metadata):
        return {
            'Type': 'Condition',
            'Formula': '$a.valueSearch(isGreaterThan(20kW))',
            'Formula Parameters': {
                '$a': self.Power()
            }
        }
    
    @Asset.Attribute()
    def Other_Compressors_Are_High_Power(self, metadata):
        # This is a more complex example of using self.all_assets() where we want to
        # look at sibling assets as opposed to parents/children, and do a roll up.
        # Here the "if" statement selects Compressor assets where our parent and
        # their parent are the same but we exclude ourselves.
        return ItemGroup([
            asset.High_Power() for asset in self.all_assets()
            if isinstance(asset, Compressor) and self.parent == asset.parent and self != asset
        ]).roll_up('union')
        
        

build_df = spy.assets.build(Refrigerator, metadata_df, errors='raise')

spy.push(metadata=build_df, workbook='SPy Documentation Examples >> spy.assets')

,Description,Type,Datasource Name,Archived,Compressor,Referenced Name,Reference,Name,Asset,Asset Object,...,Formula Parameters,Cache Enabled,Referenced ID,Scoped To,Datasource Class,Datasource ID,Data ID,ID,Push Result,Value Unit Of Measure
0,NaN,CalculatedSignal,Example Data,False,Compressor 2,Area B_Compressor Power,True,Power,Compressor 2,Refrigerator Units >> Refrigerator 1 >> Compre...,...,[signal=0EF58FC2-85A1-7590-AA5F-C66CA82059F8],False,0EF58FC2-85A1-7590-AA5F-C66CA82059F8,0EF5BE2D-2923-FB30-A567-172B344E74EE,Seeq Data Lab,Seeq Data Lab,[0EF5BE2D-2923-FB30-A567-172B344E74EE] {Signal...,0EF5BE2D-682D-6650-BABC-30A0A1EC1AA3,Success,NaN
1,NaN,CalculatedCondition,NaN,NaN,NaN,NaN,NaN,High Power,Compressor 2,Refrigerator Units >> Refrigerator 1 >> Compre...,...,[a=0EF5BE2D-682D-6650-BABC-30A0A1EC1AA3],NaN,NaN,0EF5BE2D-2923-FB30-A567-172B344E74EE,Seeq Data Lab,Seeq Data Lab,[0EF5BE2D-2923-FB30-A567-172B344E74EE] {Condit...,0EF5BE2D-6A94-EA10-A877-914BA01F8514,Success,NaN
2,NaN,CalculatedCondition,NaN,NaN,NaN,NaN,NaN,Other Compressors Are High Power,Compressor 1,Refrigerator Units >> Refrigerator 1 >> Compre...,...,[p0=0EF5BE2D-6A94-EA10-A877-914BA01F8514],False,NaN,0EF5BE2D-2923-FB30-A567-172B344E74EE,Seeq Data Lab,Seeq Data Lab,[0EF5BE2D-2923-FB30-A567-172B344E74EE] {Condit...,0EF5BE2D-6BA6-7110-8901-7F36D75CBA80,Success,NaN
3,NaN,CalculatedSignal,Example Data,False,Compressor 1,Area A_Compressor Power,True,Power,Compressor 1,Refrigerator Units >> Refrigerator 1 >> Compre...,...,[signal=0EF58FC2-8C7F-6290-BEE8-1BE39219249E],False,0EF58FC2-8C7F-6290-BEE8-1BE39219249E,0EF5BE2D-2923-FB30-A567-172B344E74EE,Seeq Data Lab,Seeq Data Lab,[0EF5BE2D-2923-FB30-A567-172B344E74EE] {Signal...,0EF5BE2D-6860-EAA0-869B-86B9099656F5,Success,NaN
4,NaN,CalculatedCondition,NaN,NaN,NaN,NaN,NaN,High Power,Compressor 1,Refrigerator Units >> Refrigerator 1 >> Compre...,...,[a=0EF5BE2D-6860-EAA0-869B-86B9099656F5],NaN,NaN,0EF5BE2D-2923-FB30-A567-172B344E74EE,Seeq Data Lab,Seeq Data Lab,[0EF5BE2D-2923-FB30-A567-172B344E74EE] {Condit...,0EF5BE2D-6ABB-FB10-A137-40F1F47D73B9,Success,NaN
5,NaN,Asset,NaN,NaN,NaN,NaN,NaN,Compressor 1,Compressor 1,Refrigerator Units >> Refrigerator 1 >> Compre...,...,NaN,NaN,NaN,0EF5BE2D-2923-FB30-A567-172B344E74EE,Seeq Data Lab,Seeq Data Lab,[0EF5BE2D-2923-FB30-A567-172B344E74EE] {Asset}...,0EF5BE2D-6979-66D0-98FA-B99C0592B0D1,Success,NaN
6,NaN,CalculatedSignal,Example Data,False,NaN,Area A_Temperature,True,Temperature,Refrigerator 1,Refrigerator Units >> Refrigerator 1,...,[signal=0EF58FC2-8206-77F0-8519-ECD251B886A6],False,0EF58FC2-8206-77F0-8519-ECD251B886A6,0EF5BE2D-2923-FB30-A567-172B344E74EE,Seeq Data Lab,Seeq Data Lab,[0EF5BE2D-2923-FB30-A567-172B344E74EE] {Signal...,0EF5BE2D-688F-60D0-8C89-B90FF76F7294,Success,NaN
7,NaN,CalculatedCondition,NaN,NaN,NaN,NaN,NaN,Compressor High Power,Refrigerator 1,Refrigerator Units >> Refrigerator 1,...,"[p0=0EF5BE2D-6A94-EA10-A877-914BA01F8514, p1=0...",NaN,NaN,0EF5BE2D-2923-FB30-A567-172B344E74EE,Seeq Data Lab,Seeq Data Lab,[0EF5BE2D-2923-FB30-A567-172B344E74EE] {Condit...,0EF5BE2D-6BCD-6210-9C54-A3A276144B1C,Success,NaN
8,NaN,CalculatedCondition,NaN,NaN,NaN,NaN,NaN,Other Compressors Are High Power,Compressor 2,Refrigerator Units >> Refrigerator 1 >> Compre...,...,[p0=0EF5BE2D-6ABB-FB10-A137-40F1F47D73B9],False,NaN,0EF5BE2D-2923-FB30-A567-172B344E74EE,Seeq Data Lab,Seeq Data Lab,[0EF5BE2D-2923-FB30-A567-172B344E74EE] {Condit...,0EF5BE2D-6BF9-7130-A144-B1BED9CE8B6C,Success,NaN
9,NaN,Asset,NaN,NaN,NaN,NaN,NaN,Compressor 2,Compressor 2,Refrigerator Units >> Refrigerator 1 >> Compre...,...,NaN,NaN,NaN,0EF5BE2D-2923-FB30-A567-172B344E74EE,Seeq Data Lab,Seeq Data Lab,[0EF5BE2D-2923-FB30-A567-172B344E74EE] {Asset}...,0EF5BE2D-6976-FFC0-80E7-2FC1F0A40968,Success,NaN


There should now be a `Refrigerator Units` asset tree in Seeq with two Refrigerators with two Compressors each.

In this manner, you can make an arbitrarily deep hierarchy composed of various asset types.

## Metrics

Metrics (aka _Scorecard Metrics_ / _Threshold Metrics_) are powerful items in Seeq that allow you to easily specify aggregations, statistics and associated boundaries. Click on `Scorecard Metric` in the Seeq Workbench _Tools_ pane to experiment with them.

Metrics can be specified as attributes in asset classes and can refer to other attributes. Here are some examples:

In [13]:
class HVAC_With_Metrics(HVAC):
    @Asset.Attribute()
    def Too_Humid(self, metadata):
        return {
            'Type': 'Condition',
            'Name': 'Too Humid',
            'Formula': '$relhumid.valueSearch(isGreaterThan(70%))',
            'Formula Parameters': {
                '$relhumid': self.Relative_Humidity(),
            }
        }

    @Asset.Attribute()
    def Humidity_Upper_Bound(self, metadata):
        return {
            'Type': 'Signal',
            'Name': 'Humidity Upper Bound',
            'Formula': '$relhumid + 10',
            'Formula Parameters': {
                '$relhumid': self.Relative_Humidity(),
            }
        }

    @Asset.Attribute()
    def Humidity_Lower_Bound(self, metadata):
        return {
            'Type': 'Signal',
            'Name': 'Humidity Lower Bound',
            'Formula': '$relhumid - 10',
            'Formula Parameters': {
                '$relhumid': self.Relative_Humidity(),
            }
        }

    @Asset.Attribute()
    def Humidity_Statistic_KPI(self, metadata):
        return {
            'Type': 'Metric',
            'Measured Item': self.Relative_Humidity(),
            'Statistic': 'Range'
        }

    @Asset.Attribute()
    def Humidity_Simple_KPI(self, metadata):
        return {
            'Type': 'Metric',
            'Measured Item': self.Relative_Humidity(),
            'Thresholds': {
                'HiHi': self.Humidity_Upper_Bound(),
                'LoLo': self.Humidity_Lower_Bound()
            }
        }

    @Asset.Attribute()
    def Humidity_Condition_KPI(self, metadata):
        return {
            'Type': 'Metric',
            'Measured Item': self.Relative_Humidity(),
            'Statistic': 'Maximum',
            'Bounding Condition': self.Too_Humid(),
            'Bounding Condition Maximum Duration': '30h'
        }

    @Asset.Attribute()
    def Humidity_Continuous_KPI(self, metadata):
        return {
            'Type': 'Metric',
            'Measured Item': self.Relative_Humidity(),
            'Statistic': 'Minimum',
            'Duration': '6h',
            'Period': '4h',
            'Metric Neutral Color':'#189E4D',
            #hex color codes can be optionally appended to thresholds
            'Thresholds': {
                'HiHiHi#FF0000': 60,
                'HiHi': 40,
                'LoLo#0000ff': 20
            }
        }

Now let's push a small asset tree with these metrics in it:

In [14]:
metadata_df = spy.search({
    'Name': 'Area A_*',
    'Type':'StoredSignal',
    'Datasource Name':'Example Data'
})

metadata_df['Build Asset'] = 'Metrics Area A'
metadata_df['Build Path'] = 'Metrics Example'
build_df = spy.assets.build(HVAC_With_Metrics, metadata_df)
push_df = spy.push(metadata=build_df, workbook='SPy Documentation Examples >> spy.assets')

You should now see a `Metrics Example` tree that you can drill into and bring up metrics.

## Troubleshooting / Debugging

Since we are using Python classes to describe our asset tree and the attributes therein, troubleshooting our code is a little bit harder than just working with DataFrames. If the code within your `@Asset.Attribute`-decorated function isn't working the way you expect, ideally you would be able to see what's happening inside of it while the `spy.assets.build()` function is running.

A useful tool is Python's built-in command-line debugger called [pdb](https://docs.python.org/3/library/pdb.html). Let's try using it in the example below. When you execute the following cell, you'll notice that an `ipdb>` prompt appears, showing you the line of code you're about to execute. You can show the value of the `metadata` variable just by typing `metadata` and pressing ENTER. Then type `c` and hit ENTER to allow execution to continue.

In [15]:
# You must import the IPython breakpoint function, called "set_trace"
from IPython.core.debugger import set_trace

class DebuggingExample(Asset):
    @Asset.Attribute()
    def My_Scalar(self, metadata):
        # Put in a "breakpoint" so that execution stops here and a command line pops up. Notice the
        # if statement here as an example of how to be more precise about when the breakpoint is hit:
        # We're only going to enter the debugger if the asset's name is 'Debugging Area A'
        if self.definition['Name'] == 'Debugging Area A':
            set_trace()
        
        return {
            'Type': 'Scalar',
            'Formula': '%s' % len(metadata)
        }
    
debugging_metadata_df = pd.DataFrame([{
    'Build Asset': 'Debugging Area A',
    'Build Path': 'Debugging Example'
}])

build_df = spy.assets.build(DebuggingExample, debugging_metadata_df)

,Build Path,Build Asset,Build Template,Build Result
0,Debugging Example,Debugging Area A,DebuggingExample,Success


There are lots of great commands to help you navigate through your code as it executes. Read through all the commands at [pdb](https://docs.python.org/3/library/pdb.html) to familiarize yourself. The most important ones are `step`, `next` and `continue`. Anything else you type at the prompt gets evaluated as Python code, so you can do things like `metadata['Build Asset']` to show just the `Build Asset` column of the `metadata` DataFrame.

### Using an Integrated Development Environment (IDE)

If you want to "level up" and start using more powerful development and debugging tools, you can! There are several good choices available to you, including [SPyder](https://www.spyder-ide.org/) (free and open source) and [PyCharm](https://www.jetbrains.com/pycharm/) (Community Edition is free).

If you take this route, you'll want to move your code into "normal" .py files and execute SPy commands from a main script with the debugger engaged.

## Detailed Help

All SPy functions have detailed documentation to help you use them. Just execute `help(spy.<func>)` like
you see below.

**Make sure you re-execute the cell below to see the latest documentation. It otherwise might be from an
earlier version of SPy.**

In [16]:
help(spy.assets.build)

Help on function build in module seeq.spy.assets._build:

build(model, metadata, *, workbooks=None, errors=None, quiet=None, status: 'Status' = None)
    Utilizes a Python Class-based asset model specification to produce a set
    of item definitions as a metadata DataFrame.
    
    Parameters
    ----------
    model : {ModuleType, type, List[type]}
        A Python module, a spy.assets.Asset or list of spy.asset.Assets to
        use as the model for the asset tree to be produced. Follow the
        spy.assets.ipynb Tutorial to understand the structure of your
        module/classes.
    
    metadata : {pd.DataFrame}
        The metadata DataFrame, usually produced from calls to spy.search(),
        that will be used as the "ingredients" for the asset tree and passed
        into all Asset.Attribute() and Asset.Component() decorated class
        functions.
    
    workbooks : {list}
        The set of workbooks (usually a single AnalysisTemplate object and
        one or more To